# Module 2.4: SNN-MPR Comparison


New data frame to 
- merge optimized_amplitude and rand_init files wrt target frequencies
- save the full parameter set
- save the full optimization information
- save error (MPR frequency - SNN frequency)/target_frequency *100


SNN files are available at 

https://www.dropbox.com/scl/fo/vv7lx16wdb3tcnn125ar2/ACWugwhwcVYstGZ81TcVIWQ?rlkey=xo57pfu62afvy75ts7makyr11&e=1&st=3x9bk27n&dl=0

Until 16/02/2026 MPR files are available at 

https://filesender.renater.fr/?s=download&token=5a124153-45f2-429d-8732-79e867588592

## 0.1 Import all built-in packages.

In [ ]:
# Import necessary packages
# %matplotlib inline
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks
import numpy as np
import pandas as pd
import pickle
import zipfile
import seaborn as sns
from pathlib import Path


In [ ]:
### Compute Power Spectrum of simulated MPR, output data.

def compute_PS_MPR(sol):
    dt_min = np.min(np.diff(sol.t))
    Fs = int(1000 / np.min(np.diff(sol.t)))
    sampling_rate_max = 1 / dt_min             # maximum sampling frequency (Hz)
    t_regular = np.arange(0, sol.t[-1], dt_min)
    y_regular = sol.sol(t_regular)
    # # Compute the Fourier Transform of the signal R_E
    sign = y_regular[0, int((t_regular[-1]-1000)/dt_min):int(t_regular[-1]/dt_min)]
    fft_values = np.fft.fft(sign)
    # # Compute the Power Spectrum (Magnitude squared of FFT)
    power_spectrum = np.abs(fft_values) ** 2
    # # Create the frequency axis
    frequencies = np.fft.fftfreq(len(sign), 1 / Fs)
    # Only keep the positive frequencies (due to symmetry)
    positive_freq_indices = frequencies > 0
    frequencies_exc = frequencies[positive_freq_indices]
    power_spectrum_exc = power_spectrum[positive_freq_indices]

    # # Compute the Fourier Transform of the signal R_I
    sign = y_regular[0, int((t_regular[-1]-1000)/dt_min):int(t_regular[-1]/dt_min)]
    fft_values = np.fft.fft(sign)
    # # Compute the Power Spectrum (Magnitude squared of FFT)
    power_spectrum = np.abs(fft_values) ** 2
    # # Create the frequency axis
    frequencies = np.fft.fftfreq(len(sign), 1 / Fs)
    # Only keep the positive frequencies (due to symmetry)
    positive_freq_indices = frequencies > 0
    frequencies_inh = frequencies[positive_freq_indices]
    power_spectrum_inh = power_spectrum[positive_freq_indices]

    return frequencies_exc, frequencies_inh, power_spectrum_exc, power_spectrum_inh

def compute_PS_SNN(data_SNN, cutoff_max_freq = 200):

    # get the frequency and power spectrum data
    freq_SNN_exc = data_SNN['fw_exc'][0:cutoff_max_freq]
    freq_SNN_inh = data_SNN['fw_inh'][0:cutoff_max_freq]

    PS_SNN_exc = data_SNN['fA_exc'][0:cutoff_max_freq]
    PS_SNN_inh = data_SNN['fA_inh'][0:cutoff_max_freq]

    PS_SNN_exc_scaled = PS_SNN_exc - np.min(PS_SNN_exc)
    PS_SNN_exc_scaled /= np.max(PS_SNN_exc_scaled)

    PS_SNN_inh_scaled = PS_SNN_inh - np.min(PS_SNN_inh)
    PS_SNN_inh_scaled /= np.max(PS_SNN_inh_scaled)

    return freq_SNN_exc, freq_SNN_inh, PS_SNN_exc_scaled, PS_SNN_inh_scaled


### Find Peaks of Power Spectrum 
def find_peaks_PS(frequencies, power_spectrum, target_freq, isitrange=True):
    if not isitrange:
        peak_frequencies = frequencies[np.argmax(power_spectrum)]
        return peak_frequencies
    else:
        ### compute the error between the target frequency and the peak frequency
        peak_frequencies = frequencies[np.argmax(power_spectrum)]

        ### compute the error in the range of the target frequency
        # Find all local maxima first
        vmin, vmax = target_freq *0, target_freq*1.90

        peaks = find_peaks(power_spectrum)

        # Filter by value range
        mask = (frequencies[peaks[0]] >= vmin) & (frequencies[peaks[0]] <= vmax)
        filtered_peaks = peaks[0][mask]
        if len(filtered_peaks) > 0:
            peak_frequencies = frequencies[filtered_peaks[np.argmax(power_spectrum[filtered_peaks])]]
        else:
            peak_frequencies = 0.0  # or some other default value if no peaks found in range

    return peak_frequencies

# 1.1 Read optimized parameters

In [ ]:
list_cols = {0:'a',
            1: 'b',
            2: 'c',
            3: 'ep_delta',
            4: 'ep_eta',
            5: 'ep_alpha',
            6: 'ep_beta',
            7: 'ep_ujump',
            8: 'ep_input',
            9: 'ip_delta',
            10: 'ip_eta',
            11: 'ip_alpha',
            12: 'ip_beta',
            13: 'ip_ujump',
            14: 'ip_input',
            15: 'cp_ee',
            16: 'cp_ii',
            17: 'cp_ie',
            18: 'cp_ei',
            19: 'sp_ge',
            20: 'sp_gi',
            21: 'sp_sejump',
            22: 'sp_sijump',
            23: 'sp_reve',
            24: 'sp_revi',
            25: 'sp_tause',
            26: 'sp_tausi',
            27: 'f_target',
            28:'fmax_MPR_exc',
            29:'fmax_SNN_exc',
            30:'error_freq_exc',
            31:'fmax_MPR_inh',
            32:'fmax_SNN_inh',
            33:'error_freq_inh',
            34: 'parameter_set_file',
            35: 'param_ID'
             }

# create data frame from list_cols
df = pd.DataFrame(columns=list_cols.values())

# 1.2 Read data, compute error, define the list

In [ ]:
target_freq = 60 # Target frequency in Hz
from pathlib import PurePosixPath
import re

def _numeric_suffix_key(fname: str):
    m = re.search(r'(\d+)(?=\.pkl$)', fname)
    return int(m.group(1)) if m else float('inf')

for parameter_set_file in ['optimized_frequencies', 'rand_init_optimization_results']:

    data_parameter_set = pd.read_csv(parameter_set_file + '.csv')

    # path to SNN
    zip_path = Path('/Users/koksal/Documents/Projects/2024_EITN/followup/codes/MPR-SNN_Data_Validation/Raw_SNN_from_'+parameter_set_file)
    zip_files = list(zip_path.glob("*.zip"))
    # group the ones starting with SNN_Data_Directory_Target+str(target_freq)
    zip_files_target = [z for z in zip_files if f'SNN_Data_Directory_Target_{target_freq}' in z.name]
    print(f"SNN Found {len(zip_files_target)} zip files for target frequency {target_freq} Hz.")
    print(zip_files_target)

    # path to MPR
    zip_path_MPR = Path('/Users/koksal/Documents/Projects/2024_EITN/followup/codes/MPR-SNN_Data_Validation/Raw_MPR_from_'+parameter_set_file)
    zip_files_MPR = list(zip_path_MPR.glob("*.zip"))
    zip_files_MPR_target = [z for z in zip_files_MPR if f'MPR_Data_Directory_Target_{target_freq}' in z.name]
    print(f"MPR Found {len(zip_files_MPR_target)} zip files for target frequency {target_freq} Hz.")
    print(zip_files_MPR_target)
    
    # # Open the zip file
    with zipfile.ZipFile(zip_files_MPR_target[0], 'r') as zip_file_MPR,\
        zipfile.ZipFile(zip_files_target[0], 'r') as zip_file:
        # pkl_file_names_SNN = [PurePosixPath(name).name for name in zip_file.namelist() if name.endswith('.pkl')]
        # pkl_file_names_MPR = [PurePosixPath(name).name for name in zip_file_MPR.namelist() if name.endswith('.pkl')]
        pkl_file_names_SNN = sorted([PurePosixPath(n).name for n in zip_file.namelist() if n.endswith('.pkl')], key=_numeric_suffix_key)
        # pkl_file_names_MPR = sorted([PurePosixPath(n).name for n in zip_file_MPR.namelist() if n.endswith('.pkl')], key=_numeric_suffix_key)
        pkl_file_names_MPR = sorted([n for n in zip_file_MPR.namelist() if n.endswith('.pkl')], key=_numeric_suffix_key) # it is because the folder zipped in OS


        # Open the pickle file inside the zip
        for i in range(len(data_parameter_set[data_parameter_set['desired_frequency']==target_freq])):
            param_fit_index = data_parameter_set.index[data_parameter_set['desired_frequency']==target_freq][i]
            with zip_file.open(pkl_file_names_SNN[i]) as pkl_file:
                data_SNN = pickle.load(pkl_file)
            with zip_file_MPR.open(pkl_file_names_MPR[i]) as pkl_file:
                data_MPR = pickle.load(pkl_file)

            # compute power spectrum for MPR
            frequencies_exc_MPR, frequencies_inh_MPR, power_spectrum_exc_MPR, power_spectrum_inh_MPR = compute_PS_MPR(data_MPR['solution'])
            # compute power spectrum for SNN
            frequencies_exc_SNN, frequencies_inh_SNN, power_spectrum_exc_SNN, power_spectrum_inh_SNN = compute_PS_SNN(data_SNN)
            # compute peak frequencies
            peak_freq_MPR_exc = find_peaks_PS(frequencies_exc_MPR, power_spectrum_exc_MPR, target_freq, isitrange=True)
            peak_freq_SNN_exc = find_peaks_PS(frequencies_exc_SNN, power_spectrum_exc_SNN, target_freq, isitrange=True)
            peak_freq_MPR_inh = find_peaks_PS(frequencies_inh_MPR, power_spectrum_inh_MPR, target_freq, isitrange=True)
            peak_freq_SNN_inh = find_peaks_PS(frequencies_inh_SNN, power_spectrum_inh_SNN, target_freq, isitrange=True)
            # compute the error between the target frequency and the peak frequency
            error_MPR_SNN_exc = (peak_freq_MPR_exc - peak_freq_SNN_exc)/target_freq * 100
            error_MPR_SNN_inh = (peak_freq_MPR_inh - peak_freq_SNN_inh)/target_freq * 100


            list_params_compare_freq = list(data_SNN['parameters'].values())
            list_params_compare_freq.append(target_freq)
            list_params_compare_freq.append(peak_freq_MPR_exc)
            list_params_compare_freq.append(peak_freq_SNN_exc)
            list_params_compare_freq.append(error_MPR_SNN_exc)
            list_params_compare_freq.append(peak_freq_MPR_inh)
            list_params_compare_freq.append(peak_freq_SNN_inh)
            list_params_compare_freq.append(error_MPR_SNN_inh)
            list_params_compare_freq.append(parameter_set_file)
            list_params_compare_freq.append(param_fit_index)
            df.loc[len(df)] = list_params_compare_freq

df.head()

In [ ]:
dataframe_file_name = '/Users/koksal/Documents/Projects/2024_EITN/followup/codes/MPR-SNN_Data_Validation/param_freq_target_'+str(target_freq)+'.csv'
df.to_csv(dataframe_file_name)

- Load optimized parameter sets
- create a list with the fitting performance
- Load "param_freq_target_X.csv"
- Add the fitting performance. 
- save as "param_freq_target_X.csv"

In [ ]:
for target_freq in [6, 10, 20, 60]:

    # Collect fitness entries from both parameter sets
    fitness_frames = []
    for parameter_set_file in ['optimized_frequencies', 'rand_init_optimization_results']:

        data_parameter_set = pd.read_csv(parameter_set_file + '.csv')
        # Select rows matching target_freq and keep 'fitness' as a DataFrame
        fitness = (data_parameter_set.loc[data_parameter_set['desired_frequency'] == target_freq, ['fitness']].reset_index(drop=True))
        fitness_frames.append(fitness)
    new_df = pd.concat(fitness_frames, axis=0, ignore_index=True)

    dataframe_file_name = '/Users/koksal/Documents/Projects/2024_EITN/followup/codes/MPR-SNN_Data_Validation/param_freq_target_'+str(target_freq)+'.csv'

    df = pd.read_csv(dataframe_file_name)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df = pd.concat([df, new_df], axis=1)
    df.to_csv(dataframe_file_name)
